# **Ejercicio 2.6**

Una cadena hotelera tiene dos buses para recoger y dejar personas en un aeropuerto local y dos hoteles separados. Los buses viajan desde el aeropuerto al hotel 1, luego al hotel 2, y regresan al aeropuerto para continuar con este patrón. El tiempo de viaje entre cada lugar sigue una distribución normal con una media de 20 y una desviación estándar de 2 minutos. El tiempo de llegada de los viajeros desde sus vuelos se distribuye exponencialmente con una media de 2.5 minutos. Cincuenta por ciento de las personas se bajan en el primer hotel, y el bus recoge personas de este hotel que desean ir al aeropuerto. El otro cincuenta por ciento de las personas se baja en el segundo hotel, y el bus recoge nuevamente personas. En el aeropuerto, todo el mundo se baja. En ambos hoteles las personas llegan al paradero del bus para ir al aeropuerto con tiempos entre llegadas exponenciales con media de 5 minutos. Simular el sistema donde el primer bus sale del aeropuerto al iniciar la simulación y el segundo sale del aeropuerto 30 minutos después del primero. Determine la cantidad de asientos requeridos en ambos buses tal que cualquier persona esperando pueda ser recogida.


Instalar Dependencias


In [1]:
pip install simpy

##CÓDIGO IMPLEMENTACIÓN EJERCICIO 2.6 SIMPY##

In [2]:
import simpy
import random
from collections import deque

SIM_TIME = 60*24
MEAN_ARRIVAL_TIME_AIRPORT = 2.5
MEAN_ARRIVAL_TIME_HOTEL = 5
MEAN_BUS_TIME_STATIONS = 20
STD_BUS_TIME = 2
BUS_START_DELAYS = [0,30]

def arrival_airport_p(env, airport_to_hotel1, airport_to_hotel2):
    alternate_destination = True
    while True:
        yield env.timeout(random.expovariate(1 / MEAN_ARRIVAL_TIME_AIRPORT))
        if alternate_destination:
            airport_to_hotel1.append(1)
        else:
            airport_to_hotel2.append(1)
        alternate_destination = not alternate_destination

def arrival_hotel_p(env, cola_h):
    while True:
        yield env.timeout(random.expovariate(1 / MEAN_ARRIVAL_TIME_HOTEL))
        cola_h.append(1)

def board_passengers(queue, max_board):
    boarded = 0
    while queue and boarded < max_board:
        queue.popleft()
        boarded += 1
    return boarded

def update_load(boarded, onboard, occupancy, space):
    onboard += boarded
    occupancy += boarded
    space -= boarded
    return onboard, occupancy, space    #onboard,occupancy,space updated

def unload_passengers(onboard, occupancy):
    occupancy = max(occupancy - onboard, 0)
    return 0, occupancy  # onboard reset, occupancy updated

def bus_trips(env, id,start_delay, capacity, airport_to_hotel1, airport_to_hotel2,
              hotel1_to_airport, hotel2_to_airport, failures,queue_access):
    yield env.timeout(start_delay)

    onboard_H1 = onboard_H2 = 0
    onboard_A = 0
    occupancy = 0

    while True:
        space = capacity

        #Exclusive access to the queues
        with queue_access.request() as req:
            yield req

        # 1. Pick up passengers in airport
        total_air_h = len(airport_to_hotel1) + len(airport_to_hotel2)

        if total_air_h > space:
            failures.append(f"{env.now:.2f} - {id} left people waiting at Airport")
        to_board_min_H1 = min(space // 2, len(airport_to_hotel1))

        boarded_H1 = board_passengers(airport_to_hotel1, to_board_min_H1)
        onboard_H1, occupancy, space = update_load(boarded_H1, onboard_H1, occupancy, space)

        to_board_min_H2 = min(space, len(airport_to_hotel2))
        boarded_H2 = board_passengers(airport_to_hotel2, to_board_min_H2)
        onboard_H2, occupancy, space = update_load(boarded_H2, onboard_H2, occupancy, space)


        # 2. Airport → Hotel 1
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard_H1, occupancy = unload_passengers(onboard_H1, occupancy)
        space = capacity - occupancy

        with queue_access.request() as req:
                yield req

        if len(hotel1_to_airport) > space:
            failures.append(f"{env.now:.2f} - {id} left people waiting at Hotel 1")
        boarded_from_H1 = board_passengers(hotel1_to_airport, space)
        onboard_A, occupancy, space = update_load(boarded_from_H1, onboard_A, occupancy, space)


        # 3. Hotel 1 → Hotel 2
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard_H2, occupancy = unload_passengers(onboard_H2, occupancy)
        space = capacity - occupancy

        with queue_access.request() as req:
                yield req
        if len(hotel2_to_airport) > space:
            failures.append(f"{env.now:.2f} - {id} left people waiting at Hotel 2")
        boarded_from_H2 = board_passengers(hotel2_to_airport, space)
        onboard_A, occupancy, space = update_load(boarded_from_H2, onboard_A, occupancy, space)


        # 4. Hotel 2 → Airport
        with queue_access.request() as req:
                yield req
        yield env.timeout(max(random.gauss(MEAN_BUS_TIME_STATIONS, STD_BUS_TIME), 0.1))
        onboard_A,occupancy = unload_passengers(onboard_A,occupancy)

def simulation(capacity):
        env = simpy.Environment()
        airport_to_hotel1, airport_to_hotel2 = deque(), deque()
        hotel1_to_airport, hotel2_to_airport = deque(), deque()
        failures = []

        #Shared resource to avoid interference
        queue_access = simpy.Resource(env, capacity=1)

        env.process(arrival_airport_p(env, airport_to_hotel1, airport_to_hotel2))
        env.process(arrival_hotel_p(env, hotel1_to_airport))
        env.process(arrival_hotel_p(env, hotel2_to_airport))
        env.process(bus_trips(env, 'Bus1', BUS_START_DELAYS[0],capacity, airport_to_hotel1, airport_to_hotel2, hotel1_to_airport, hotel2_to_airport, failures,queue_access))
        env.process(bus_trips(env, 'Bus2', BUS_START_DELAYS[1],capacity, airport_to_hotel1, airport_to_hotel2, hotel1_to_airport, hotel2_to_airport, failures,queue_access))

        env.run(until=SIM_TIME)
        return failures

def minimum_capacity(rep=1000, max_cap=100):
    for capacity in range(10, max_cap + 1):
        sim_success = 0
        for _ in range(rep):
            failures = simulation(capacity)
            if not failures:
                sim_success += 1
        print(f"Capacity for {capacity} passengers: {sim_success / rep * 100:.2f}% success rate")
        if sim_success == rep:
            return capacity
    return None


print("Minimum capacity per bus:", minimum_capacity())

Capacity for 10 passengers: 0.00% success rate
Capacity for 11 passengers: 0.00% success rate
Capacity for 12 passengers: 0.00% success rate
Capacity for 13 passengers: 0.00% success rate
Capacity for 14 passengers: 0.00% success rate
Capacity for 15 passengers: 0.00% success rate
Capacity for 16 passengers: 0.00% success rate
Capacity for 17 passengers: 0.00% success rate
Capacity for 18 passengers: 0.00% success rate
Capacity for 19 passengers: 0.30% success rate
Capacity for 20 passengers: 1.00% success rate
Capacity for 21 passengers: 2.60% success rate
Capacity for 22 passengers: 5.40% success rate
Capacity for 23 passengers: 10.40% success rate
Capacity for 24 passengers: 17.50% success rate
Capacity for 25 passengers: 23.60% success rate
Capacity for 26 passengers: 36.40% success rate
Capacity for 27 passengers: 44.00% success rate
Capacity for 28 passengers: 49.60% success rate
Capacity for 29 passengers: 64.80% success rate
Capacity for 30 passengers: 72.40% success rate
Capac

##CÓDIGO IMPLEMENTACIÓN EJERCICIO 2.6 SIMLIB##

In [3]:
%%writefile simlib.h
#ifndef SIMLIB_H
    #define SIMLIB_H

extern float sim_time;
extern int event_type;

void init_simlib();
void event_schedule(float time, int event_type);
void timing();
float expon(float mean, int stream);
float normal(float mean, float stddev);

#endif


Writing simlib.h


In [4]:
%%writefile simlib.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <limits.h>
#include <float.h>
#include "simlib.h"

#define MAX_EVENTS 1000

typedef struct {
    float time;
    int type;
} Event;

Event event_list[MAX_EVENTS];
int num_events = 0;
float sim_time = 0.0;
int event_type;

void init_simlib() {
    num_events = 0;
    sim_time = 0.0;
}

void event_schedule(float time, int type) {
    if (num_events >= MAX_EVENTS) {
        fprintf(stderr, "Too many events\n");
        exit(1);
    }
    event_list[num_events].time = time;
    event_list[num_events].type = type;
    num_events++;
}

void timing() {
    int min_index = -1;
    float min_time = FLT_MAX;

    for (int i = 0; i < num_events; i++) {
        if (event_list[i].time < min_time) {
            min_time = event_list[i].time;
            min_index = i;
        }
    }

    if (min_index == -1) {
        fprintf(stderr, "No events left\n");
        exit(1);
    }

    sim_time = event_list[min_index].time;
    event_type = event_list[min_index].type;

    // remove the event
    for (int i = min_index; i < num_events - 1; i++) {
        event_list[i] = event_list[i + 1];
    }
    num_events--;
}

float expon(float mean, int stream) {
    return -mean * log(1.0 - ((float)rand() / RAND_MAX));
}

float normal(float mean, float stddev) {
    static int haveSpare = 0;
    static double spare;

    if (haveSpare) {
        haveSpare = 0;
        return mean + stddev * spare;
    }

    haveSpare = 1;
    double u, v, s;
    do {
        u = ((double)rand() / RAND_MAX) * 2 - 1;
        v = ((double)rand() / RAND_MAX) * 2 - 1;
        s = u * u + v * v;
    } while (s >= 1 || s == 0);

    s = sqrt(-2.0 * log(s) / s);
    spare = v * s;
    return mean + stddev * (u * s);
}


Writing simlib.c


In [5]:
%%writefile simulationpastor.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include <time.h>
#include "simlib.h"

#define SIM_TIME (60 * 24)
#define MEAN_ARRIVAL_AIRPORT 2.5
#define MEAN_ARRIVAL_HOTEL 5
#define MEAN_BUS_TIME 20
#define STD_BUS_TIME 2
#define BUS_START_DELAYS_1 0
#define BUS_START_DELAYS_2 30
#define MAX_QUEUE 10000

#define EVENT_ARRIVAL_AIRPORT 1
#define EVENT_ARRIVAL_HOTEL1 2
#define EVENT_ARRIVAL_HOTEL2 3
#define EVENT_BUS1 4
#define EVENT_BUS2 5
#define EVENT_END 6

void advance_time(int id);

int airport_to_hotel1 = 0, airport_to_hotel2 = 0;
int hotel1_to_airport = 0, hotel2_to_airport = 0;
int failures = 0;
int capacity;

void arrival_airport() {
    if (airport_to_hotel1 <= airport_to_hotel2)
        airport_to_hotel1++;
    else
        airport_to_hotel2++;

    event_schedule(sim_time + expon(MEAN_ARRIVAL_AIRPORT, 1), EVENT_ARRIVAL_AIRPORT);
}

void arrival_hotel1() {
    hotel1_to_airport++;
    event_schedule(sim_time + expon(MEAN_ARRIVAL_HOTEL, 2), EVENT_ARRIVAL_HOTEL1);
}

void arrival_hotel2() {
    hotel2_to_airport++;
    event_schedule(sim_time + expon(MEAN_ARRIVAL_HOTEL, 3), EVENT_ARRIVAL_HOTEL2);
}

void bus_trip(int bus_id) {
    int onboard_H1 = 0, onboard_H2 = 0, onboard_A = 0, occupancy = 0;
    int space, boarded;

    space = capacity;

    // Airport pickup
    if ((airport_to_hotel1 + airport_to_hotel2) > space)
        failures++;

    boarded = (space / 2 < airport_to_hotel1) ? space / 2 : airport_to_hotel1;
    airport_to_hotel1 -= boarded;
    onboard_H1 += boarded;
    occupancy += boarded;
    space -= boarded;

    boarded = (space < airport_to_hotel2) ? space : airport_to_hotel2;
    airport_to_hotel2 -= boarded;
    onboard_H2 += boarded;
    occupancy += boarded;
    space -= boarded;

    // Airport to Hotel 1
    advance_time(bus_id);
    occupancy -= onboard_H1;
    onboard_H1 = 0;
    space = capacity - occupancy;

    if (hotel1_to_airport > space)
        failures++;

    boarded = (space < hotel1_to_airport) ? space : hotel1_to_airport;
    hotel1_to_airport -= boarded;
    onboard_A += boarded;
    occupancy += boarded;
    space -= boarded;

    // Hotel 1 to Hotel 2
    advance_time(bus_id);
    occupancy -= onboard_H2;
    onboard_H2 = 0;
    space = capacity - occupancy;

    if (hotel2_to_airport > space)
        failures++;

    boarded = (space < hotel2_to_airport) ? space : hotel2_to_airport;
    hotel2_to_airport -= boarded;
    onboard_A += boarded;
    occupancy += boarded;
    space -= boarded;

    // Hotel 2 to Airport
    advance_time(bus_id);
    occupancy -= onboard_A;
    onboard_A = 0;

    // Reprogramar el mismo bus para el siguiente viaje
    event_schedule(sim_time, (bus_id == 1) ? EVENT_BUS1 : EVENT_BUS2);
}

void advance_time(int id) {
    float t = normal(MEAN_BUS_TIME, STD_BUS_TIME);
    if (t < 0.1) t = 0.1;
    sim_time += t;
}

int simulation(int cap) {
    srand(time(NULL) + rand());  // Semilla distinta para cada simulación
    init_simlib();
    sim_time = 0.0;
    capacity = cap;
    airport_to_hotel1 = airport_to_hotel2 = 0;
    hotel1_to_airport = hotel2_to_airport = 0;
    failures = 0;

    event_schedule(expon(MEAN_ARRIVAL_AIRPORT, 1), EVENT_ARRIVAL_AIRPORT);
    event_schedule(expon(MEAN_ARRIVAL_HOTEL, 2), EVENT_ARRIVAL_HOTEL1);
    event_schedule(expon(MEAN_ARRIVAL_HOTEL, 3), EVENT_ARRIVAL_HOTEL2);
    event_schedule(BUS_START_DELAYS_1, EVENT_BUS1);
    event_schedule(BUS_START_DELAYS_2, EVENT_BUS2);

    while (sim_time < SIM_TIME) {
        timing();
        switch (event_type) {
            case EVENT_ARRIVAL_AIRPORT: arrival_airport(); break;
            case EVENT_ARRIVAL_HOTEL1: arrival_hotel1(); break;
            case EVENT_ARRIVAL_HOTEL2: arrival_hotel2(); break;
            case EVENT_BUS1: bus_trip(1); break;
            case EVENT_BUS2: bus_trip(2); break;
        }
    }

    return failures;
}

int main() {
    int rep = 1000;
    int max_cap = 100;
    int cap;

    for (cap = 10; cap <= max_cap; cap++) {
        int success = 0;
        for (int i = 0; i < rep; i++) {
            if (simulation(cap) == 0)
                success++;
        }
        printf("Capacity %d: %.2f%% success rate\n", cap, (100.0 * success) / rep);
        if (success == rep) {
            printf("Minimum capacity per bus: %d\n", cap);
            break;
        }
    }

    return 0;
}


Writing simulationpastor.c


In [6]:
!gcc simulationpastor.c simlib.c -o simulacion -lm
!./simulacion

Capacity 10: 0.00% success rate
Capacity 11: 0.00% success rate
Capacity 12: 0.00% success rate
Capacity 13: 0.00% success rate
Capacity 14: 0.00% success rate
Capacity 15: 0.00% success rate
Capacity 16: 0.00% success rate
Capacity 17: 0.00% success rate
Capacity 18: 0.00% success rate
Capacity 19: 0.20% success rate
Capacity 20: 0.80% success rate
Capacity 21: 2.70% success rate
Capacity 22: 7.50% success rate
Capacity 23: 12.30% success rate
Capacity 24: 19.60% success rate
Capacity 25: 27.10% success rate
Capacity 26: 36.80% success rate
Capacity 27: 42.50% success rate
Capacity 28: 52.00% success rate
Capacity 29: 61.00% success rate
Capacity 30: 70.20% success rate
Capacity 31: 79.20% success rate
Capacity 32: 84.60% success rate
Capacity 33: 89.90% success rate
Capacity 34: 92.40% success rate
Capacity 35: 96.40% success rate
Capacity 36: 98.50% success rate
Capacity 37: 98.60% success rate
Capacity 38: 99.00% success rate
Capacity 39: 99.50% success rate
Capacity 40: 99.70% suc